In [ ]:
%pip install opentrons paho-mqtt

  Using cached anyio-3.7.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (114 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.6/905.6 kB 29.4 MB/s eta 0:00:00
Using cached anyio-3.7.1-py3-none-any.whl (80 kB)
Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.1
    Uninstalling numpy-2.0.1:
      Successfully uninstalled numpy-2.0.1
  Attempting uninstall: anyio━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/8 [numpy]
    Found existing installation: anyio 4.9.0━━━━━━━━━━━━━━━━━━ 2/8 [numpy]
    Uninstalling anyio-4.9.0:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/8 [numpy]
      Successfully uninstalled anyio-4.9.0━━━━━━━━━━━━━━━━━━━━ 2/8 [numpy]
  Attempting uninstall: jsonschema━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/8 [numpy]
    Found existing installation: jsonschema 4.23.0━━━━━━━━━━━━ 2/

In [2]:
# source: https://support.opentrons.com/s/article/Running-the-robot-using-Jupyter-Notebook

# import opentrons.execute
import opentrons.simulate  # if you want to simulate the protocol first

# This is where you establish the API version for executing a protocol
# protocol = opentrons.execute.get_protocol_api('2.11')
# to simulate the protocol, replace opentrons.execute with opentrons.simulate
# like so
protocol = opentrons.simulate.get_protocol_api("2.16")

# Must home before running
protocol.home()

# Labware
plate = protocol.load_labware("nest_96_wellplate_200ul_flat", "2")

tiprack_1 = protocol.load_labware("opentrons_96_tiprack_1000ul", location="1")

# Pipette
p1000 = protocol.load_instrument("p1000_single_gen2", "right", tip_racks=[tiprack_1])

# Protocol steps
p1000.transfer(100, plate["A1"], plate["A2"])

# Output steps in protocol
for line in protocol.commands():
    print(line)

/Users/pod_0x000d/.opentrons/robot_settings.json not found. Loading defaults
Deck calibration not found.
/Users/pod_0x000d/.opentrons/deck_calibration.json not found. Loading defaults


Transferring 100.0 from A1 of NEST 96 Well Plate 200 µL Flat on slot 2 to A2 of NEST 96 Well Plate 200 µL Flat on slot 2
Picking up tip from A1 of Opentrons OT-2 96 Tip Rack 1000 µL on slot 1
Aspirating 100.0 uL from A1 of NEST 96 Well Plate 200 µL Flat on slot 2 at 274.7 uL/sec
Dispensing 100.0 uL into A2 of NEST 96 Well Plate 200 µL Flat on slot 2 at 274.7 uL/sec
Dropping tip into Trash Bin on slot 12


In [3]:
!git clone https://github.com/AccelerationConsortium/ac-training-lab.git

Cloning into 'ac-training-lab'...
remote: Enumerating objects: 4142, done.
remote: Counting objects: 100% (993/993), done.
remote: Compressing objects: 100% (294/294), done.
remote: Total 4142 (delta 782), reused 731 (delta 669), pack-reused 3149 (from 1)
Receiving objects: 100% (4142/4142), 142.26 MiB | 3.36 MiB/s, done.
Resolving deltas: 100% (2282/2282), done.


In [4]:
%cd /content/ac-training-lab/src/ac_training_lab/ot-2/_scripts/

[Errno 2] No such file or directory: '/content/ac-training-lab/src/ac_training_lab/ot-2/_scripts/'
/Users/pod_0x000d/Documents/GitHub/ac-training-lab/src/ac_training_lab/ot-2


In [5]:
import json
from queue import Empty, Queue
from time import sleep

# import opentrons.execute
import opentrons.simulate
import paho.mqtt.client as mqtt

# protocol = opentrons.execute.get_protocol_api("2.16")
protocol = opentrons.simulate.get_protocol_api("2.16")

OT2_SERIAL = "OT2CEP20240218R04"
PICO_ID = "e66130100f895134"

# MQTT Broker Configuration
host = "248cc294c37642359297f75b7b023374.s2.eu.hivemq.cloud"
username = "sgbaird"
password = "D.Pq5gYtejYbU#L"
port = 8883

OT2_COMMAND_TOPIC = f"command/ot2/{OT2_SERIAL}/pipette"
OT2_STATUS_TOPIC = f"status/ot2/{OT2_SERIAL}/complete"
# SENSOR_COMMAND_TOPIC = f"command/picow/{PICO_ID}/as7341/read"
# SENSOR_DATA_TOPIC = f"color-mixing/picow/{PICO_ID}/as7341"

# Initialize MQTT client and sensor data queue
client = mqtt.Client()
client.tls_set(tls_version=mqtt.ssl.PROTOCOL_TLS_CLIENT)  # type: ignore
client.username_pw_set(username, password)

command_queue = Queue()


# MQTT Callbacks
def on_connect(client, userdata, flags, rc):
    print("Connected to MQTT Broker with result code", rc)
    client.subscribe(OT2_COMMAND_TOPIC, qos=2)


def on_message(client, userdata, msg):
    payload = msg.payload.decode("utf-8")
    print(f"Received message on topic {msg.topic}: {payload}")
    try:
        payload = json.loads(payload)
        if msg.topic == OT2_COMMAND_TOPIC:
            command_queue.put(payload)

    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON payload: {e}")


client.on_connect = on_connect
client.on_message = on_message
client.connect(host, port)
client.loop_start()

print("MQTT client connected")

protocol.home()  # home to know MQTT cilent is connected
# Define labware and pipettes

# changed file path before nohup run
# load wireless charging port
with open(
    # "/var/lib/jupyter/notebooks/ac_color_sensor_charging_port.json"
    "ac_color_sensor_charging_port.json"
) as labware_file1:
    labware_def1 = json.load(labware_file1)
    tiprack_2 = protocol.load_labware_from_definition(labware_def1, 10)
# load 3x2 vials rack
with open("ac_6_tuberack_15000ul.json") as labware_file2:
    # with open("/var/lib/jupyter/notebooks/ac_6_tuberack_15000ul.json") as labware_file2:
    labware_def2 = json.load(labware_file2)
    reservoir = protocol.load_labware_from_definition(labware_def2, 3)
# load other labwares from Opentrons's labware library
plate = protocol.load_labware(load_name="corning_96_wellplate_360ul_flat", location=1)

tiprack_1 = protocol.load_labware(load_name="opentrons_96_tiprack_300ul", location=9)

p300 = protocol.load_instrument(
    instrument_name="p300_single_gen2", mount="right", tip_racks=[tiprack_1]
)

p300.well_bottom_clearance.dispense = 8


print("Labwares loaded")


# this includes a 3-color mix and a measurement
def mix_color(payload):
    R = payload["command"]["R"]
    Y = payload["command"]["Y"]
    B = payload["command"]["B"]
    mix_well = payload["command"]["well"]
    session_id = payload["session_id"]
    experiment_id = payload["experiment_id"]

    total = R + Y + B
    if total > 300:
        raise ValueError("The sum of the proportions must be not greater than 300")

    position = [
        "B1",
        "B2",
        "B3",
    ]  # R,Y,B paint vial on the position of the 3x2 vials rack
    portion = {"B1": R, "B2": Y, "B3": B}
    # total_volume = 280
    red_volume = R  # int(portion["B1"] * total_volume)
    yellow_volume = Y  # int(portion["B2"] * total_volume)
    blue_volume = B  # int(portion["B3"] * total_volume)
    color_volume = {"B1": red_volume, "B2": yellow_volume, "B3": blue_volume}

    for pos in position:
        if float(portion[pos]) != 0.0:  # if zero, skip this color aspiration process
            p300.pick_up_tip(tiprack_1[pos])
            p300.aspirate(color_volume[pos], reservoir[pos])
            p300.dispense(color_volume[pos], plate[mix_well])
            p300.default_speed = 100
            # reduce pipette speed to prevent droplets falling, speed unit: mm/sec
            p300.blow_out(reservoir["A1"].top(z=-5))
            p300.default_speed = (
                400  # reset pipette speed, speed unit: mm/sec, 400 is default value
            )
            p300.drop_tip(tiprack_1[pos])

    p300.pick_up_tip(tiprack_2["A2"])
    p300.move_to(
        plate[mix_well].top(z=-1.3)
    )  # z=-1.3 to get sensor closer to the well, can change this value depends on
    # the fitting of the pick-up fake tip on the sensor package.

    # payloadtosent = json.dumps(payload)
    # print("Sending read command to sensor...")
    # client.publish(SENSOR_COMMAND_TOPIC, payloadtosent, qos=2)

    print("Sending status to HF...")
    payload_data = {
        "status": {"sensor_status": "in_place"},
        "experiment_id": experiment_id,
        "session_id": session_id,
    }
    payload = json.dumps(payload_data)
    client.publish(OT2_STATUS_TOPIC, payload, qos=2)  # send a status message back to HF


def move_sensor_back(payload):
    results_status = payload["command"]["sensor_status"]
    session_id = payload["session_id"]
    experiment_id = payload["experiment_id"]

    p300.drop_tip(tiprack_2["A2"].top(z=-80))
    # protocol.home()

    payload_data = {
        "status": {"sensor_status": "charging"},
        "experiment_id": experiment_id,
        "session_id": session_id,
    }
    payload = json.dumps(payload_data)
    client.publish(OT2_STATUS_TOPIC, payload, qos=2)  # send a status message back to HF

    if results_status == "sensor_timeout":
        protocol.home()


def handle_command(payload):

    if {"R", "Y", "B", "well"}.issubset(payload["command"].keys()):
        print(f"Handling mix command: {payload}")
        mix_color(payload)

    elif {"sensor_status"}.issubset(payload["command"].keys()):
        print("Sensor measure complete")
        move_sensor_back(payload)


print("OT-2 is waiting for command")
protocol.home()  # home to know OT-2 is ready


# Keep protocol active
while True:
    try:
        command = command_queue.get(timeout=1)
        print(f"Processing command from queue: {command}")

        if "command" in command and "experiment_id" in command:
            try:
                handle_command(command)
            except Exception as e:
                print(f"Error processing command: {e}")
    except Empty:

        pass

    except Exception as e:

        print(f"Unexpected error in main loop: {e}")
    sleep(1)

/Users/pod_0x000d/.opentrons/robot_settings.json not found. Loading defaults
Deck calibration not found.
/Users/pod_0x000d/.opentrons/deck_calibration.json not found. Loading defaults
/var/folders/wr/_t87_gmn2vgcds3jc_6s_rvh0000gn/T/ipykernel_14275/155514568.py:27: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


MQTT client connected
Connected to MQTT Broker with result code 0


FileNotFoundError: [Errno 2] No such file or directory: 'ac_color_sensor_charging_port.json'

## Code Graveyard

In [ ]:
# %%writefile protocol_gui.py

# https://docs.opentrons.com/v2/

# from opentrons import protocol_api

# # metadata
# metadata = {
#     "protocolName": "My Protocol",
#     "author": "Name <opentrons@example.com>",
#     "description": "Simple protocol to get started using the OT-2",
# }

# # requirements
# requirements = {"robotType": "OT-2", "apiLevel": "2.22"}

# # protocol run function
# def run(protocol: protocol_api.ProtocolContext):
#     # labware
#     plate = protocol.load_labware(
#         "corning_96_wellplate_360ul_flat", location="1"
#     )
#     tiprack = protocol.load_labware(
#         "opentrons_96_tiprack_300ul", location="2"
#     )

#     # pipettes
#     left_pipette = protocol.load_instrument(
#         "p300_single", mount="left", tip_racks=[tiprack]
#     )

#     # commands
#     left_pipette.pick_up_tip()
#     left_pipette.aspirate(100, plate["A1"])
#     left_pipette.dispense(100, plate["B2"])
#     left_pipette.drop_tip()

# # Usage: `!opentrons_simulate protocol_gui.py`

In [ ]:
# %%writefile protocol_execute.py
# import opentrons.execute

# protocol = opentrons.execute.get_protocol_api("2.16")

In [ ]:
# > !opentrons_simulate protocol_execute.py

# Traceback (most recent call last):
#   File "/usr/local/bin/opentrons_simulate", line 8, in <module>
#     sys.exit(main())
#              ^^^^^^
#   File "/usr/local/lib/python3.11/dist-packages/opentrons/simulate.py", line 1022, in main
#     runlog, maybe_bundle = simulate(
#                            ^^^^^^^^^
#   File "/usr/local/lib/python3.11/dist-packages/opentrons/simulate.py", line 536, in simulate
#     protocol = parse.parse(
#                ^^^^^^^^^^^^
#   File "/usr/local/lib/python3.11/dist-packages/opentrons/protocols/parse.py", line 346, in parse
#     return _parse_python(
#            ^^^^^^^^^^^^^^
#   File "/usr/local/lib/python3.11/dist-packages/opentrons/protocols/parse.py", line 254, in _parse_python
#     version = _get_version(static_info, parsed, ast_filename)
#               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#   File "/usr/local/lib/python3.11/dist-packages/opentrons/protocols/parse.py", line 571, in _get_version
#     raise MalformedPythonProtocolError(
# opentrons.protocols.types.MalformedPythonProtocolError: apiLevel not declared in protocol_execute.py. You must specify the target API version in the apiLevel key of the metadata dict. For instance, metadata={"apiLevel": "2.0"}